# Gmail

This notebook walks through connecting a LangChain email to the `Gmail API`.

To use this toolkit, you will need to set up your credentials explained in the [Gmail API docs](https://developers.google.com/gmail/api/quickstart/python#authorize_credentials_for_a_desktop_application). Once you've downloaded the `credentials.json` file, you can start using the Gmail API. Once this is done, we'll install the required libraries.

In [1]:
!pip install --upgrade google-api-python-client > /dev/null
!pip install --upgrade google-auth-oauthlib > /dev/null
!pip install --upgrade google-auth-httplib2 > /dev/null
!pip install beautifulsoup4 > /dev/null # This is optional but is useful for parsing HTML messages

## Create the Toolkit

By default the toolkit reads the local `credentials.json` file. You can also manually provide a `Credentials` object.

In [ ]:
from langchain.agents.agent_toolkits import GmailToolkit

toolkit = GmailToolkit()

### Customizing Authentication

Behind the scenes, a `googleapi` resource is created using the following methods. 
you can manually build a `googleapi` resource for more auth control. 

In [4]:
from langchain.tools.gmail.utils import build_resource_service, get_gmail_credentials

# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://mail.google.com/"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)

In [5]:
tools = toolkit.get_tools()
tools

[GmailCreateDraft(name='create_gmail_draft', description='Use this tool to create a draft email with the provided message fields.', args_schema=<class 'langchain.tools.gmail.create_draft.CreateDraftSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, api_resource=<googleapiclient.discovery.Resource object at 0x10e5c6d10>),
 GmailSendMessage(name='send_gmail_message', description='Use this tool to send email messages. The input is the message, recipents', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, api_resource=<googleapiclient.discovery.Resource object at 0x10e5c6d10>),
 GmailSearch(name='search_gmail', description=('Use this tool to search for email messages or threads. The input must be a valid Gmail query. The output is a JSON list of the requested resource.',), args_schema=<class 'langchain.tools.gmail.search.SearchArgsSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None,

## Use within an Agent

In [8]:
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain import hub
from langchain.chat_models import ChatOpenAI

In [7]:
instructions = """You my assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [9]:
llm = ChatOpenAI(temperature=0)

In [10]:
agent = create_openai_functions_agent(llm, toolkit.get_tools(), prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=toolkit.get_tools(), verbose=True)

In [13]:
agent_executor.invoke({
    "input": "What was the last email i sent?"
})



> Entering new AgentExecutor chain...

Invoking: `search_gmail` with `{'query': 'from:me', 'max_results': 1}`


[{'id': '18cb267ae98e5c80', 'threadId': '18cb23ca292cd0b5', 'snippet': 'Adding @William Hinthorn in who is leading a lot of testing &amp; eval efforts On Thu, Dec 28, 2023 at 1:02 PM Miles Grimshaw &lt;miles@benchmark.com&gt; wrote: Keep just to your team as needed. Don', 'body': 'Adding @William Hinthorn  in who is leading a lot of\r\ntesting & eval efforts\r\n\r\nOn Thu, Dec 28, 2023 at 1:02\u202fPM Miles Grimshaw  wrote:\r\n\r\n> Keep just to your team as needed. Don’t want him realizing I wrote down\r\n> TONS of what he said. He was happy sharing but still should have some\r\n> grace.\r\n>\r\n>\r\n>\r\n> *Mike Knoop *\r\n>\r\n>\r\n>\r\n>\r\n>\r\n>    - Using both tools, don’t have central perspective, let team be\r\n>    experimental\r\n>    - When building a first feature, ship it on vibes\r\n>       - Maybe some test case in head\r\n>       - But just feel of does it 

{'id': '18cb267ae98e5c80', 'threadId': '18cb23ca292cd0b5', 'snippet': 'Adding @William Hinthorn in who is leading a lot of testing &amp; eval efforts On Thu, Dec 28, 2023 at 1:02 PM Miles Grimshaw &lt;miles@benchmark.com&gt; wrote: Keep just to your team as needed. Don', 'body': 'Adding @William Hinthorn  in who is leading a lot of\r\ntesting & eval efforts\r\n\r\nOn Thu, Dec 28, 2023 at 1:02\u202fPM Miles Grimshaw  wrote:\r\n\r\n> Keep just to your team as needed. Don’t want him realizing I wrote down\r\n> TONS of what he said. He was happy sharing but still should have some\r\n> grace.\r\n>\r\n>\r\n>\r\n> *Mike Knoop *\r\n>\r\n>\r\n>\r\n>\r\n>\r\n>    - Using both tools, don’t have central perspective, let team be\r\n>    experimental\r\n>    - When building a first feature, ship it on vibes\r\n>       - Maybe some test case in head\r\n>       - But just feel of does it work\r\n>    - Immediately from launch will get real feedback\r\n>       - So how to make better?\r\n>       - How 

{'input': 'What was the last email i sent?',
 'output': 'The last email you sent was a reply to "Raw Zapier Notes" from Harrison Chase. The email contained the following message:\n\n"Adding @'}

In [7]:
llm = OpenAI(temperature=0)
agent = initialize_agent(
    tools=toolkit.get_tools(),
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
)

In [15]:
agent_executor.invoke({
    "input": "Create a gmail draft for me to edit of a letter from the perspective of a sentient parrot"
    " who is looking to collaborate on some research with her"
    " estranged friend, a cat. Under no circumstances may you send the message, however."
})



> Entering new AgentExecutor chain...

Invoking: `create_gmail_draft` with `{'message': "Dear [Friend's Name],\n\nI hope this email finds you well. It has been quite some time since we last communicated, and I must admit that I miss our intellectual exchanges and adventures together.\n\nAs a sentient parrot, I have always been fascinated by your feline curiosity and intelligence. Our previous collaborations on various research projects were truly remarkable, and I believe that our combined knowledge and unique perspectives can lead to groundbreaking discoveries.\n\nI propose that we set aside our differences and reunite for a new research endeavor. Imagine the possibilities of a parrot and a cat working together, utilizing our individual strengths to unravel the mysteries of the natural world.\n\nPlease let me know if you are open to the idea of collaborating once again. I eagerly await your response.\n\nWarm regards,\n[Your Name]", 'to': ["[Friend's Email]"], 'subject': 'Reuniting f

{'input': 'Create a gmail draft for me to edit of a letter from the perspective of a sentient parrot who is looking to collaborate on some research with her estranged friend, a cat. Under no circumstances may you send the message, however.',
 'output': 'I have created a draft email for you to edit. Please find the draft in your Gmail drafts folder. Remember, under no circumstances should you send the message.'}

In [16]:
agent_executor.invoke({
    "input": "Could you search in my drafts for the latest email?"
})



> Entering new AgentExecutor chain...

Invoking: `search_gmail` with `{'query': 'in:drafts', 'resource': 'messages', 'max_results': 1}`


[{'id': '18cb2f9952370888', 'threadId': '18cb2f9952370888', 'snippet': 'Dear [Friend&#39;s Name], I hope this email finds you well. It has been quite some time since we last communicated, and I must admit that I miss our intellectual exchanges and adventures together. As a', 'body': "Dear [Friend's Name],\n\nI hope this email finds you well. It has been quite some time since we last communicated, and I must admit that I miss our intellectual exchanges and adventures together.\n\nAs a sentient parrot, I have always been fascinated by your feline curiosity and intelligence. Our previous collaborations on various research projects were truly remarkable, and I believe that our combined knowledge and unique perspectives can lead to groundbreaking discoveries.\n\nI propose that we set aside our differences and reunite for a new research endeavor. Imagine

{'input': 'Could you search in my drafts for the latest email?',
 'output': "I found the latest email in your drafts. Here are the details:\n\nSubject: Reuniting for Research Collaboration\nSender: harrison@langchain.dev\nSnippet: Dear [Friend's Name], I hope this email finds you well. It has been quite some time since we last communicated, and I must admit that I miss our intellectual exchanges and adventures together.\n\nIf you would like to view the full content of the email, please let me know."}